In [1]:
#Primer paso que se realizo fue importar todas las librerias que vamos a utilizar
import pandas as pd
import json
import re
from fastapi import FastAPI
from datetime import datetime

Todos los codigos necesarios para la API

In [221]:
# Crear una instancia de FastAPI
app = FastAPI()
#Formula para Director
@app.get("/director/{director}")
def get_director(director: str):
    df1 = df[df["director"].notna() &df["director"].str.contains(director)]
    df1["ganancia"] = df1["revenue"] - df1["budget"]
    retorno_director = df1["return"].sum()
    resp = df1[["title", "release_year", "return", "budget", "ganancia"]]

    return [director, retorno_director, resp]




In [222]:
#Pruebas para ver si funcionan el codigo
get_director("Howard Deutch")

C:\Users\Admin\AppData\Local\Temp\ipykernel_1908\1971259495.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["ganancia"] = df1["revenue"] - df1["budget"]


['Howard Deutch',
 inf,
                         title release_year    return      budget    ganancia
 2            Grumpier Old Men   1995-12-22       NaN         0.0         0.0
 1252   Some Kind of Wonderful   1987-02-27       inf         0.0  18553948.0
 1736        The Odd Couple II   1998-04-09       NaN         0.0         0.0
 3744         The Replacements   2000-08-07       NaN         0.0         0.0
 6716               Article 99   1992-03-13  0.354221  18000000.0 -11624021.0
 23164                 Gleason   2002-10-13       NaN         0.0         0.0]

In [223]:
## Formula para VOTOS
@app.get('/votos_titulo')
def votos_titulo(titulo_de_la_filmacion):
    pelicula = df[df["title"] == titulo_de_la_filmacion]
    
    if pelicula.empty:
        return "No se encontró la filmación en el dataset."
    
    cantidad_votos = pelicula["vote_count"].iloc[0]
    
    if cantidad_votos < 2000:
        return "La filmación no cumple con la cantidad mínima de valoraciones."
    
    promedio_votos = pelicula["vote_average"].iloc[0]
    mensaje = f"La película {titulo_de_la_filmacion} fue estrenada en el año {pelicula['release_year'].iloc[0]}. Cuenta con un total de {cantidad_votos} valoraciones, con un promedio de {promedio_votos}."
    
    return mensaje


In [224]:
votos_titulo('Jumanji')

'La película Jumanji fue estrenada en el año 1995-12-15 00:00:00. Cuenta con un total de 2413.0 valoraciones, con un promedio de 6.9.'

In [225]:
## Formula para Actor
@app.get('/get_actor')
def get_actor(nombre_actor):
    actor_df = df[df["actors"].str.contains(nombre_actor, case=False, na=False)]
    actor_df["retorno"] = actor_df["revenue"] - actor_df["budget"]
    
    cantidad_peliculas = len(actor_df)
    retorno_total = actor_df["retorno"].sum()
    promedio_retorno = actor_df["retorno"].mean()
    
    mensaje = f"El actor {nombre_actor} ha participado en {cantidad_peliculas} filmaciones. Ha conseguido un retorno total de {retorno_total} con un promedio de {promedio_retorno} por filmación."
    
    return mensaje

In [226]:
get_actor('Tom hanks')

C:\Users\Admin\AppData\Local\Temp\ipykernel_1908\2790984468.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actor_df["retorno"] = actor_df["revenue"] - actor_df["budget"]


'El actor Tom hanks ha participado en 42 filmaciones. Ha conseguido un retorno total de 6324162814.0 con un promedio de 150575305.0952381 por filmación.'

In [227]:
# Formula para Score
@app.get('/score_titulo')
def score_titulo(titulo_de_la_filmacion):
    pelicula = df[df["title"] == titulo_de_la_filmacion]
    
    if pelicula.empty:
        return "No se encontró la filmación en el dataset."
    
    titulo = pelicula["title"].iloc[0]
    ano_estreno = pelicula["release_year"].iloc[0]
    score = pelicula["popularity"].iloc[0]
    
    mensaje = f"La película {titulo} fue estrenada en el año {ano_estreno} con un score/popularidad de {score}."
    
    return mensaje


In [228]:
score_titulo('Toy Story')

'La película Toy Story fue estrenada en el año 1995-10-30 00:00:00 con un score/popularidad de 21.946943.'

In [229]:
#Formula para la cantidad de filmaciones por MES
@app.get('/cantidad_filmaciones_mes')
def cantidad_filmaciones_mes(mes):
    mes = mes.lower()
    peliculas_mes = df[df["release_month"] == mes]
    cantidad = len(peliculas_mes)
    
    mensaje = f"{cantidad} cantidad de películas fueron estrenadas en el mes {mes.capitalize()}."
    
    return mensaje

In [230]:
cantidad_filmaciones_mes('julio')

'2646 cantidad de películas fueron estrenadas en el mes Julio.'

In [231]:
#Formula para ver cuantas peliculas fueron estrenadas por cierto dia
@app.get('/cantidad_filmaciones_dia/{dia}')
def cantidad_filmaciones_dia(dia):
    dia = dia.lower()
    filmaciones_dia = df[df["dia_espanol"].str.lower() == dia]
    cantidad = len(filmaciones_dia)
    
    mensaje = f"{cantidad} cantidad de películas fueron estrenadas en el día {dia.capitalize()}."
    
    return mensaje

In [233]:
cantidad_filmaciones_dia('Martes')

'4644 cantidad de películas fueron estrenadas en el día Martes.'

In [ ]:

# Convertir columna a tipo datetime
df['release_year'] = pd.to_datetime(df['release_year'])

# Extraer el mes de la columna 'release_date'
df['release_month'] = df['release_year'].dt.month

# Mostrar el DataFrame resultante con la columna 'release_month'
print(df)


In [78]:
# Modificar los meses para la consigna
# Extraer el mes de la columna 'release_date'
df['release_month'] = df['release_year'].dt.month

In [80]:
#Cambio los numeros a su mes correspondiente
def convertir_mes_letras(numero_mes):
    meses = {
        1: "enero",
        2: "febrero",
        3: "marzo",
        4: "abril",
        5: "mayo",
        6: "junio",
        7: "julio",
        8: "agosto",
        9: "septiembre",
        10: "octubre",
        11: "noviembre",
        12: "diciembre"
    }
    
    if numero_mes in meses:
        return meses[numero_mes]
    else:
        return "Mes inválido"

# Supongamos que tienes un DataFrame llamado df con una columna "mes" que contiene números de mes
df["release_month"] = df["release_month"].map(convertir_mes_letras)

In [82]:
#Formula para crear dias
df['release_day'] = df['release_year'].dt.day

In [83]:
#Se obtiene el nombre de cada dia en formato texto
df["release_year"] = pd.to_datetime(df["release_year"])  

def obtener_nombre_dia(fecha):
    nombre_dia = fecha.strftime("%A")
    return nombre_dia

df["nombre_dia"] = df["release_year"].apply(obtener_nombre_dia)

In [84]:
#Pasamos los dias de Ingles a Español
diccionario_dias = {
    "Monday": "Lunes",
    "Tuesday": "Martes",
    "Wednesday": "Miercoles",
    "Thursday": "Jueves",
    "Friday": "Viernes",
    "Saturday": "Sábado",
    "Sunday": "Domingo"
}

df["nombre_dia_espanol"] = df["nombre_dia"].map(diccionario_dias)

In [85]:
#Borro una columna que me quedo de mas
df.drop('nombre_dia', axis=1, inplace=True)

In [89]:
#Borro columna llamada original_language
df.drop('original_language', axis=1, inplace=True)

In [135]:
#Cambie nombre release_date
df.rename(columns={'nombre_dia_espanol':'dia_espanol'}, inplace=True)

In [192]:
#Ultimo push para guardar el dataset listo
df.to_csv('Csv_Proyecto_Terminado.csv', index=False)

Funcion Recomendación

In [ ]:
## Importo Libreria
from typing import List

In [ ]:
@app.get('/recomendacion')
async def recomendacion(titulo: str) -> List[str]:
    # Convertir el título proporcionado a minúsculas
    titulo = titulo.lower()

    # Obtener la fila correspondiente al título proporcionado
    pelicula = df.loc[df['title'].str.lower() == titulo]

    if pelicula.empty:
        return []

    # Obtener la puntuación de la película
    puntuacion = pelicula['vote_average'].values[0]

    # Encontrar películas similares según la puntuación
    peliculas_similares = df.loc[df['vote_average'] >= puntuacion].sort_values('vote_average', ascending=False)

    # Obtener los títulos de las 5 películas con mayor puntuación
    recomendaciones = peliculas_similares['title'].head(5).tolist()

    return recomendaciones

if _name_ == '_main_':
    app.run()